# Tutorial for Element D2_nn8_quad

## Visualize the Element Type

This element type is a 2D rectangular element with eight nodes. There is one node at each corner of the rectangle, and one at the midpoints of each edge. Natural coordinates $(\xi, \eta)$ refer to the local coordinate system for each node; every node has the same natural coordinates. The natural coordinates for the D2_nn8_quad element are $(-1, -1)$, $(1, -1)$, $(1, 1)$, $(-1, 1)$, $(0, -1)$, $(1, 0)$, $(0, 1)$, and $(-1, 0)$.

Here, the element type, number of Gaus points, and the image file name (fname) are defined to display an image of the element in its natural coordinates. The Gauss point is an integration point, which will be explored later. 


In [ ]:
# Visualize the element type
from finiteelementanalysis import discretization_demo_helper_fcns as di_demo

ele_type = "D2_nn8_quad"
num_gauss_pts = 9
fname = "D2_nn8_quad_9gp.png"
di_demo.visualize_gauss_pts(fname, ele_type, num_gauss_pts)

## Interpolated Field

If the values of some field variable are known at the element nodes, the field can be interpolated to other points within the element. This field can be a scalar (like temperature) or a vector (like displacement).

For example, the value of an field variable can be found at each of the nodes based on the following equation
$$ f = 2\xi + 3\eta + 1.5\xi\eta$$

Where $f$ is the field variable (interpolated field) and $(\xi,\eta)$ are the natural cooridnates. The field value at each node can be found by evaluating the function for each of the nodal coordinates. Then, the field variable can be interpolated for other points in the element.

Interpolation of the field values is accomplished by using shape functions. Since interpolation is done in natural coordinates, the shape functions must also be defined in natural coordinates. The number of interpolation points along each axis is $10$ unless otherwise specified when calling the function.

In [ ]:
from finiteelementanalysis import discretization_demo_helper_fcns as di_demo
import numpy as np

def fcn(xi, eta):
    return 2.0 * xi + 3.0 * eta + 1.5 * xi * eta

ele_type = "D2_nn8_quad"
node_coords = np.array([[-1,-1], [1, -1], [1, 1],[-1,1],[0,-1],[1,0],[0,1],[-1,0]])
node_values = np.array([fcn(xi, eta) for xi, eta in node_coords])

fname = ele_type + "_interpolate_fcn.png"
di_demo.plot_interpolate_field_natural_coords_single_element(fname, ele_type, node_values)

## Plot Interpolated Field in Physical Coordinates

So far, we have only looked at plots of elements in their natural coordinates. However, the physical coordinates reflect the actual field locations and distribution. Let's consider the same field variable function, but this time it is defined in physical coordinates (x, y):
$$ f = 2x + 3y + 1.5xy$$

This function tells us the value of $f$ at the three nodes. If we define the nodal coordinates for the three nodes in physical coordinates, we can find the interpolated filed in natural coordinates, and then map this field to the physical coordinates. Here the default number of interpolation points is 20. Notice in the mapped element, the nodal locations are as defined by the user, but the interpolated field is still mapped similar to the reference element in natural coordinates.

In [ ]:
from finiteelementanalysis import discretization_demo_helper_fcns as di_demo
import numpy as np

def fcn(x, y):
    return 2.0 * x + 3.0 * y + 1.5 * x * y

ele_type = "D2_nn8_quad"
node_coords = np.array([[1, 1],[5, 0],[3,4],[0,4],[3,0.5], [4,2], [1, 4],[0.5,2.5]])
node_values = np.array([fcn(x, y) for x, y in node_coords])
fname = ele_type + "_interpolate_fcn_physical_coords.png"
di_demo.visualize_isoparametric_mapping_single_element(str(fname), ele_type, node_coords, node_values)

## Checking the Derivatives

Derivatives of the field variable function can be used for interpolation, but these derivatives should be validated. This can be done with an analytic solutionf for a simple field distribution, like the function used in the previous steps:
$$ f = 2x + 3y + 1.5xy$$

The derivative of this function can be found:
$$ \frac{\partial f}{\partial x} = 2+1.5y$$
$$ \frac{\partial f}{\partial y} = 3+1.5x$$

The mapped derivative (analytic) is calcualted based on the function derivative and the test point in physical coordinates.The physical gradient is the gradient of the function calculated in natural coordinates and then transformed into physical coordinates. These should match eachother. 

In [ ]:
from finiteelementanalysis import discretization_demo_helper_fcns as di_demo
import numpy as np

def fcn(x, y):
    return 2.0 * x + 3.0 * y + 1.5 * x * y

def fcn_deriv(x, y):
    return np.asarray([2.0 + 1.5 * y, 3.0 + 1.5 * x]).reshape((2, 1))

ele_type = "D2_nn8_quad"
node_coords = np.array([[0, 0], [4, 0], [2, 2], [0, 2],[2,0],[3,1],[1,2],[0,1]])
node_values = np.array([fcn(x, y) for x, y in node_coords])

xi, eta = 0, 0  # element center in natural coordinates

# Map the test point from natural to physical coordinates
x_mapped = di_demo.interpolate_field_natural_coords_single_element(
    ele_type, node_coords[:, 0], [xi], [eta]
).flatten()[0]
y_mapped = di_demo.interpolate_field_natural_coords_single_element(
    ele_type, node_coords[:, 1], [xi], [eta]
).flatten()[0]

# Evaluate the function derivative in physical coordinates
mapped_deriv = fcn_deriv(x_mapped, y_mapped)

# Compute the numerical gradient in natural coordinates
gradient_natural = di_demo.interpolate_gradient_natural_coords_single_element(
    ele_type, node_values, np.array([xi]), np.array([eta])
)

# Transform the numerical gradient to physical coordinates
gradient_physical = di_demo.transform_gradient_to_physical(
    ele_type, node_coords, np.array([xi]), np.array([eta]), gradient_natural
).reshape((2, 1))

print('Mapped Derivative:\n', mapped_deriv)
print('Physical Gradient:\n',gradient_physical)

if np.allclose(mapped_deriv, gradient_physical, atol=10e-10):
    print("Analytical and numerical derivatives match!")

## Integrate Over the Area

Finally, the integration of the derivatives over the element area can be performed. This means that the field variable can be interpolated to any point in the element (domain). The most efficient points to choose for integration are known as Gauss points. Similar to checking the derivaties, we can also verify that the integral is calculated correctly by comparing to an analytic solution.

In [ ]:
from finiteelementanalysis import discretization_demo_helper_fcns as di_demo
import numpy as np

def fcn(x, y):
    return 3.0 * x + 10.0 * y

def fcn_deriv(x, y):
    return np.asarray([3.0, 10.0])  # df/dx = 3.0, df/dy = 0

def element_area(node_coords):
    x1, y1 = node_coords[0]
    x2, y2 = node_coords[1]
    x3, y3 = node_coords[2]
    x4, y4 = node_coords[3]
    area = 0.5 * abs((x1 * y2 + x2 * y3 + x3 * y4 + x4 * y1) 
                     - (y1 * x2 + y2 * x3 + y3 * x4 + y4 * x1))
    return area

def integral_of_deriv(node_coords):
    area = element_area(node_coords)
    return np.array([3.0 * area, 10.0 * area]).reshape((2, 1))

ele_type = "D2_nn8_quad"
num_gauss_pts = 4
node_coords = np.array([[0, 0], [4, 0], [2, 2], [0, 2],[2,0],[3,1],[1,2],[0,1]])

# Compute nodal values from fcn(x, y)
nodal_values = np.array([[fcn(x, y) for x, y in node_coords]]).T

# Compute numerical integral
integral_numerical = di_demo.compute_integral_of_derivative(
    ele_type, num_gauss_pts, node_coords, nodal_values)

# Compute analytical integral
integral_analytical = integral_of_deriv(node_coords)

print('Numerical Integral:\n', integral_numerical)
print('Analytic Integral:\n',integral_analytical)

# check result
if np.allclose(integral_numerical, integral_analytical, atol=10e-10):
    print("analytical and numerical integrals of derivatives match!")